In [ ]:
import requests as re
from requests.exceptions import ReadTimeout

from bs4 import BeautifulSoup
import json

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [32]:
base_url = "https://gujarati.oneindia.com/"

#categories = {'news':1, 'news/city':1, 'movies':0, 'astrology':0, 'auto':0,
#             'travel':0}

In [ ]:
try:
    with open("base_url.json", 'r') as f:
        base_url_dict = json.load(f)
except Exception as e:
    base_url_dict = {}

headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
for category in categories.keys():

    if categories[category] == 1:
        continue

    flag = 1
    page = 1
    count = 0
    prev_articles = []
    while(flag):

        url = base_url+category+'//?page-no='+str(page)
        response = re.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        articles = soup.find_all('div', class_='oi-article-thumb')
        if (prev_articles == articles) or (len(articles)==0):
            if count>=10:
                break
            page+=1
            count+=1
            print(f"Count: {count} Page:{page}")
            prev_articles = articles
            continue
        prev_articles = articles

        for article in articles:

            a = article.find('a')
            base_url_dict[a['href']] = 0

        
        page+=1
        print(f"Count: {count} Page:{page}")

    categories[category] = 1
    print(category)
    with open("base_url.json", 'w') as f:
        json.dump(base_url_dict, f, indent=4)

In [ ]:
with open('./base_url.json', 'r') as f:

    url_dict = json.load(f)

try:
    text_df = pd.read_csv('./oneindia_text.csv')

except FileNotFoundError:
    text_df = pd.DataFrame(columns=['Link','Text'])

for idx, url in enumerate(url_dict.keys()):

    if url_dict[url] == 1:
        continue

    url_ = "https://gujarati.oneindia.com" + url

    text = []
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = re.get(url_, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    x = soup.find_all('p')
    for p in x:  
        text.append(p.get_text())
    
    text_df.loc[len(text_df),:] = [url_, text]
    url_dict[url] = 1

    if idx%100 == 0:
        print(idx)
        with open('./base_url.json', 'w') as f:

            json.dump(url_dict, f, indent=4)

        text_df.to_csv("./oneindia_text.csv", index=False)

In [9]:
text_df.to_csv("./oneindia_text.csv", index=False)